In [1]:
import pandas as pd
import csv
import selenium
from selenium import webdriver

from requests import get

import requests
from webdrivermanager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException

from collections import defaultdict
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
from dateutil import parser
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [2]:
bunq_comments = 'https://together.bunq.com/all'
OUTPUT_CSV = 'bunq_together.csv'

In [3]:

def scrape_bunq_comments_250(soup):
    #scrape links containing comments
    
    comments_urls = []    
    

    url = soup
    driver = webdriver.Chrome()

    driver.get(url)
    
    date = driver.find_elements_by_xpath('//*[@id="content"]/div/div[3]/div/div/div[2]/ul/li/div/div/a[2]/ul/li/span/time')
    
    #Use last date of the page to control by time which comments are scrapped

    last_date = date[-1].get_attribute('datetime')
    last_date_f= parser.parse(last_date)
    
    #Use selenium click() to keep loading comments until the desired date

    while last_date_f > parser.parse('2021-07-01T18:00:52+02:00'):
        date1 = driver.find_elements_by_xpath('//*[@id="content"]/div/div[3]/div/div/div[2]/ul/li/div/div/a[2]/ul/li/span/time')
        last_date = date1[-1].get_attribute('datetime')
        last_date_f= parser.parse(last_date)

        button = driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div/div[2]/div/button')
        button.click()
        #Necesary for page to load
        wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[3]/div/div/div[2]/div/button')))
        
    links = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/div/div/ul/li/div/div/a[2]')
    
    for element in links:
        comments_urls.append(element.get_attribute('href'))
     
        
    

    driver.quit()
    
    return comments_urls

In [4]:

def scrape_comment_page(dom):
    # scrape data contained in the comment page, resuting in a dictionary of data
    
    all_data = {}
    
    all_data['Users']= []
    all_data['Title']= []
    all_data['Time']= []
    all_data['Comments']= []
    all_data['Time_per_comment']= []

    driver = webdriver.Chrome()
    driver.get(dom)
    wait = WebDriverWait(driver, 10)
    
    
    
    users = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/div/div/div/article/div/header/ul/li/div/h3/a')
    time = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/div/div/div/article/div/header/ul/li/div/a/time')
    title = driver.find_element_by_xpath('//*[@id="content"]/div/div/header/div/ul/li/h2')
    
    # Actual Comments conatin 2 descriptions in order to select just the desired comment loop through it
    x=0
    stop = 0
    while True:
        x += 1
        try:
            text = driver.find_element_by_xpath(f'//*[@id="content"]/div/div/div/div/div/div[{x}]/article/div/div/p[2]')
        
            all_data['Comments'].append(text.text)
        except NoSuchElementException:
            try:
                text = driver.find_element_by_xpath(f'//*[@id="content"]/div/div/div/div/div/div[{x}]/article/div/div/p')
                all_data['Comments'].append(text.text)
                
            except NoSuchElementException:
                stop += 1
                if stop > 2:
                    break
                    
    
    for element in users:
        all_data["Users"].append(element.text)
    for element in time:
        all_data['Time_per_comment'].append(element.get_attribute('datetime'))
    all_data['Title'].append(title.text)
    all_data['Time'].append(time[-1].get_attribute('datetime'))
    
    driver.quit()
    
    return all_data

In [5]:

def data(dom):
    url_strings = scrape_bunq_comments_250(dom)

    rows = []
    for i, url in enumerate(url_strings): 
        rows.append(scrape_comment_page(url))
    
    return rows

full_data = data(bunq_comments)

In [11]:
def save_csv(filename, rows):
    # create csv file
    
    csv_columns = ['Users','Title','Time','Time_per_comment','Comments']    
    with open(filename, 'w', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        writer.writerows(rows)
                

In [12]:
save_csv(OUTPUT_CSV, full_data)

In [13]:
df = pd.read_csv (OUTPUT_CSV)
df

,Users,Title,Time,Time_per_comment,Comments
0,['bunq'],['Local Currencies BETA 💰'],['2021-08-31T19:24:17+02:00'],['2021-08-31T19:24:17+02:00'],['You have the unique opportunity to be one of...
1,['bunq'],['bunq Update 18 FAQ 📚'],['2021-08-31T18:50:59+02:00'],['2021-08-31T18:50:59+02:00'],"['With the exciting bunq Update 18, here comes..."
2,['bunq'],['bunq Update 18 Overview ✨'],['2021-08-31T18:14:31+02:00'],['2021-08-31T18:14:31+02:00'],['Let’s take a look at everything that’s new!']
3,['bunq'],['Phishing and Spoofing - The new ways of scam...,['2021-08-24T15:40:49+02:00'],['2021-08-24T15:40:49+02:00'],['Our legal department has provided us with a ...
4,['bunq'],['Easily rent a car with your bunq credit card...,['2021-08-20T16:22:56+02:00'],['2021-08-20T16:22:56+02:00'],['Lots of you are using rental car services. W...
...,...,...,...,...,...
635,"['Graad', 'Peter B', 'Markus']",['Afschrift Creditcard'],['2021-06-29T12:16:06+02:00'],"['2021-06-29T11:15:37+02:00', '2021-06-29T12:0...",['Hoe download ik de afschrift van mijn credit...
636,"['Davoxx', 'Markus']",['Sofortüberweisung'],['2021-06-29T08:03:22+02:00'],"['2021-06-29T06:55:10+02:00', '2021-06-29T08:0...",['Mir ist gestern folgendes aufgefallen:\nWenn...
637,"['Bernard', 'Jakob']",['Kosten geld opnemen'],['2021-06-29T01:04:51+02:00'],"['2021-06-28T15:56:49+02:00', '2021-06-29T01:0...","['Bedankt', 'Previously\n10x month free, after..."
638,"['Samuel van Daalen', 'Jakob', 'Samuel van Daa...",['How can I add a personal cardholder to a Tra...,['2021-06-29T00:54:32+02:00'],"['2021-06-28T08:12:21+02:00', '2021-06-28T15:3...","['When adding a Travel Card, I can only choose..."
